### Backtrader多数据画图

In [2]:
from qmtbt import QMTStore
from datetime import datetime
import backtrader as bt
import pandas as pd
import efinance
import matplotlib.pyplot as plt

plt.style.use("default")


def get_k_data(stock_code, begin: datetime, end: datetime) -> pd.DataFrame:
    """根据efinance工具包获取股票数据
    :param stock_code:股票代码
    :param begin: 开始日期
    :param end: 结束日期
    """
    # stock_code = '600519'  # 股票代码，茅台
    k_dataframe: pd.DataFrame = efinance.stock.get_quote_history(
        stock_code, beg=begin.strftime("%Y%m%d"), end=end.strftime("%Y%m%d")
    )
    k_dataframe = k_dataframe.iloc[:, :9]
    k_dataframe.columns = [
        "name",
        "code",
        "date",
        "open",
        "close",
        "high",
        "low",
        "volume",
        "turnover",
    ]
    k_dataframe.index = pd.to_datetime(k_dataframe.date)
    k_dataframe.drop(["name", "code", "date"], axis=1, inplace=True)
    return k_dataframe


class SmaCross(bt.SignalStrategy):
    def __init__(self):
        sma1, sma2 = bt.ind.SMA(period=10), bt.ind.SMA(period=30)
        crossover = bt.ind.CrossOver(sma1, sma2)
        self.signal_add(bt.SIGNAL_LONG, crossover)


class MultiSMACrossStrategy(bt.Strategy):
    params = (("fast_length", 5), ("slow_length", 25))

    def __init__(self):
        self.crossovers = []

        for d in self.datas:
            ma_fast = bt.ind.SMA(d, period=self.params.fast_length)
            ma_slow = bt.ind.SMA(d, period=self.params.slow_length)

            self.crossovers.append(bt.ind.CrossOver(ma_fast, ma_slow))

    def next(self):
        for i, d in enumerate(self.datas):
            if not self.getposition(d).size:
                if self.crossovers[i] > 0:
                    self.buy(data=d, size=100)
            elif self.crossovers[i] < 0:
                if self.getposition(d).size > 0:
                    self.close(data=d)


class PrintData(bt.Strategy):
    def next(self):
        for data in self.datas:
            print(f"Data for {data._name}:")
            print(f"Date: {data.datetime.date(0)}")
            print(
                f"Open: {data.open[0]}, High: {data.high[0]}, Low: {data.low[0]},"
                f" Close: {data.close[0]}, Volume: {data.volume[0]}"
            )
            print("-" * 40)


if __name__ == "__main__":

    # 获取数据
    start_time = datetime(2020, 1, 1)
    end_time = datetime(2021, 1, 1)
    cerebro = bt.Cerebro()

    # dataframe = get_k_data('000001', begin=start_time, end=end_time)
    # data = bt.feeds.PandasData(dataname=dataframe, fromdate=start_time, todate=end_time)
    store = QMTStore()
    code_list = ["000001.SZ", "600519.SH", "000002.SZ"]
    datas = store.getdatas(
        code_list=code_list,
        timeframe=bt.TimeFrame.Days,
        fromdate=datetime(2020, 1, 1),
        todate=datetime(2021, 1, 1),
        live=False,
    )

    for data in datas:
        cerebro.adddata(data)
    cerebro.addstrategy(MultiSMACrossStrategy)
    start_cash = 1000000
    cerebro.broker.setcash(start_cash)
    cerebro.broker.setcommission(commission=0.00025)
    cerebro.run()
    # cerebro.plot(iplot=False)

    # print(cerebro.runstrats[0])
    print(cerebro.broker.getvalue())

下载数据000001.SZ
000001.SZ历史数据装载成功！
下载数据600519.SH
600519.SH历史数据装载成功！
下载数据000002.SZ
000002.SZ历史数据装载成功！
1059701.0686667003


### 多线程

In [ ]:
from datetime import datetime
import backtrader as bt
import pandas as pd
import efinance
import matplotlib.pyplot as plt

plt.style.use("default")


def get_k_data(stock_code, begin: datetime, end: datetime) -> pd.DataFrame:
    """根据efinance工具包获取股票数据
    :param stock_code:股票代码
    :param begin: 开始日期
    :param end: 结束日期
    """
    # stock_code = '600519'  # 股票代码，茅台
    k_dataframe: pd.DataFrame = efinance.stock.get_quote_history(
        stock_code, beg=begin.strftime("%Y%m%d"), end=end.strftime("%Y%m%d")
    )
    k_dataframe = k_dataframe.iloc[:, :9]
    k_dataframe.columns = [
        "name",
        "code",
        "date",
        "open",
        "close",
        "high",
        "low",
        "volume",
        "turnover",
    ]
    k_dataframe.index = pd.to_datetime(k_dataframe.date)
    k_dataframe.drop(["name", "code", "date"], axis=1, inplace=True)
    return k_dataframe


class SmaCross(bt.SignalStrategy):
    def __init__(self):
        sma1, sma2 = bt.ind.SMA(period=10), bt.ind.SMA(period=30)
        crossover = bt.ind.CrossOver(sma1, sma2)
        self.signal_add(bt.SIGNAL_LONG, crossover)


class MultiSMACrossStrategy(bt.Strategy):
    params = (("fast_length", 5), ("slow_length", 25))

    def __init__(self):
        self.crossovers = []

        for d in self.datas:
            ma_fast = bt.ind.SMA(d, period=self.params.fast_length)
            ma_slow = bt.ind.SMA(d, period=self.params.slow_length)

            self.crossovers.append(bt.ind.CrossOver(ma_fast, ma_slow))

    def next(self):
        for i, d in enumerate(self.datas):
            if not self.getposition(d).size:
                if self.crossovers[i] > 0:
                    self.buy(data=d, size=100)
            elif self.crossovers[i] < 0:
                if self.getposition(d).size > 0:
                    self.close(data=d)


if __name__ == "__main__":
    store = QMTStore()
    # 获取数据
    start_time = datetime(2020, 1, 1)
    end_time = datetime(2021, 1, 1)
    cerebro = bt.Cerebro(maxcpus=10)
    # dataframe = get_k_data('000001', begin=start_time, end=end_time)
    # data = bt.feeds.PandasData(dataname=dataframe, fromdate=start_time, todate=end_time)
    # cerebro.adddata(data)
    # dataframe = get_k_data('600519', begin=start_time, end=end_time)
    # data = bt.feeds.PandasData(dataname=dataframe, fromdate=start_time, todate=end_time)
    # cerebro.adddata(data)
    # dataframe = get_k_data('000002', begin=start_time, end=end_time)
    # data = bt.feeds.PandasData(dataname=dataframe, fromdate=start_time, todate=end_time)
    code_list = ["000001.SH", "600519.SH", "000002.SH"]
    datas = store.getdatas(
        code_list=code_list,
        timeframe=bt.TimeFrame.Days,
        fromdate=datetime(2020, 1, 1),
        todate=datetime(2021, 1, 1),
        live=False,
    )
    store.setdatas(cerebro, datas)
    # cerebro.adddata(data)
    cerebro.addstrategy(MultiSMACrossStrategy)
    start_cash = 1000000
    cerebro.broker.setcash(start_cash)
    cerebro.broker.setcommission(commission=0.00025)
    cerebro.run()
    cerebro.plot(iplot=False)

    print(cerebro.runstrats[0])
    print(cerebro.broker.getvalue())

### Sko优化demo

In [1]:
from datetime import datetime
import backtrader as bt
import pandas as pd
import efinance
import optuna


def get_k_data(stock_code, begin: datetime, end: datetime) -> pd.DataFrame:
    """根据efinance工具包获取股票数据
    :param stock_code:股票代码
    :param begin: 开始日期
    :param end: 结束日期
    """
    # stock_code = '600519'  # 股票代码，茅台
    k_dataframe: pd.DataFrame = efinance.stock.get_quote_history(
        stock_code, beg=begin.strftime("%Y%m%d"), end=end.strftime("%Y%m%d")
    )
    k_dataframe = k_dataframe.iloc[:, :9]
    k_dataframe.columns = [
        "name",
        "code",
        "date",
        "open",
        "close",
        "high",
        "low",
        "volume",
        "turnover",
    ]
    k_dataframe.index = pd.to_datetime(k_dataframe.date)
    k_dataframe.drop(["name", "code", "date"], axis=1, inplace=True)
    return k_dataframe


class SmaCross(bt.SignalStrategy):
    params = (
        ("period1", 10),
        ("period2", 30),
    )

    def __init__(self):
        sma1, sma2 = bt.ind.SMA(period=self.p.period1), bt.ind.SMA(
            period=self.p.period2
        )
        crossover = bt.ind.CrossOver(sma1, sma2)
        self.signal_add(bt.SIGNAL_LONG, crossover)


start_time = datetime(2020, 1, 1)
end_time = datetime(2021, 1, 1)
# dataframe = get_k_data('600519', begin=start_time, end=end_time)
# # =============== 为系统注入数据 =================
# # 加载数据
# data = bt.feeds.PandasData(dataname=dataframe, fromdate=start_time, todate=end_time)

store = QMTStore()
code_list = ["600519.SH"]
data = store.getdata(
    dataname="600519.SH",
    timeframe=bt.TimeFrame.Days,
    fromdate=datetime(2020, 1, 1),
    todate=datetime(2021, 1, 1),
    live=False,
)


def backtest(p) -> float:
    p1, p2 = p
    strategy = SmaCross
    cerebro = bt.Cerebro()
    cerebro.adddata(data)
    cerebro.addstrategy(strategy, period1=int(p1), period2=int(p2))
    start_cash = 1000000
    cerebro.broker.setcash(start_cash)
    cerebro.broker.setcommission(commission=0.00025)
    cerebro.run()
    print(-(start_cash - cerebro.broker.getvalue()))
    return -(start_cash - cerebro.broker.getvalue())


from sko.GA import GA

ga = GA(
    func=backtest,
    n_dim=2,
    size_pop=10,
    max_iter=100,
    prob_mut=0.001,
    lb=[1, 1],
    ub=[50, 50],
    precision=1e-7,
)
best_x, best_y = ga.run()
print("best_x:", best_x, "\n", "best_y:", best_y)

# from sko.DE import DE
# de = DE(func=backtest, n_dim=2, size_pop=10, max_iter=100, lb=[1, 1], ub=[50, 50])
# best_x, best_y = de.run()
# print('best_x:', best_x, '\n', 'best_y:', best_y)

# from sko.PSO import PSO
# pso = PSO(func=backtest, n_dim=2, pop=10, max_iter=100, lb=[1, 1], ub=[50, 50], w=0.8, c1=0.5, c2=0.5)
# pso.run()
# print('best_x is ', pso.gbest_x, 'best_y is', pso.gbest_y)

NameError: name 'QMTStore' is not defined

### Optuna优化接口

In [1]:
from datetime import datetime
import backtrader as bt
import pandas as pd
import efinance
import optuna


def get_k_data(stock_code, begin: datetime, end: datetime) -> pd.DataFrame:
    """根据efinance工具包获取股票数据
    :param stock_code:股票代码
    :param begin: 开始日期
    :param end: 结束日期
    """
    # stock_code = '600519'  # 股票代码，茅台
    k_dataframe: pd.DataFrame = efinance.stock.get_quote_history(
        stock_code, beg=begin.strftime("%Y%m%d"), end=end.strftime("%Y%m%d")
    )
    k_dataframe = k_dataframe.iloc[:, :9]
    k_dataframe.columns = [
        "name",
        "code",
        "date",
        "open",
        "close",
        "high",
        "low",
        "volume",
        "turnover",
    ]
    k_dataframe.index = pd.to_datetime(k_dataframe.date)
    k_dataframe.drop(["name", "code", "date"], axis=1, inplace=True)
    return k_dataframe


class SmaCross(bt.SignalStrategy):
    params = (
        ("period1", 10),
        ("period2", 30),
    )

    def __init__(self):
        sma1, sma2 = bt.ind.SMA(period=self.p.period1), bt.ind.SMA(
            period=self.p.period2
        )
        crossover = bt.ind.CrossOver(sma1, sma2)
        self.signal_add(bt.SIGNAL_LONG, crossover)


# start_time = datetime(2020, 1, 1)
# end_time = datetime(2021, 1, 1)
# dataframe = get_k_data('600519', begin=start_time, end=end_time)
# data = bt.feeds.PandasData(dataname=dataframe, fromdate=start_time, todate=end_time)
store = QMTStore()
code_list = ["600519.SH"]
data = store.getdata(
    dataname="600519.SH",
    timeframe=bt.TimeFrame.Days,
    fromdate=datetime(2020, 1, 1),
    todate=datetime(2021, 1, 1),
    live=False,
)


def backtest(trial) -> float:
    p1 = trial.suggest_int("p1", 1, 50)
    p2 = trial.suggest_int("p2", 1, 50)
    strategy = SmaCross
    cerebro = bt.Cerebro()
    cerebro.adddata(data)
    cerebro.addstrategy(strategy, period1=int(p1), period2=int(p2))
    start_cash = 1000000
    cerebro.broker.setcash(start_cash)
    cerebro.broker.setcommission(commission=0.00025)
    cerebro.run()
    return start_cash - cerebro.broker.getvalue()


study = optuna.create_study(direction="maximize")
study.optimize(backtest, n_trials=100, n_jobs=1)
trial = study.best_trial

print("\tBest Score: {}".format(trial.value))
print("\tBest Params: {}".format(trial.params))

NameError: name 'QMTStore' is not defined

### Backtrader带log记录版本

In [16]:
import backtrader
from loguru import logger
import pandas as pd
import efinance
from datetime import datetime, timedelta


def get_k_data(stock_code, begin: datetime, end: datetime) -> pd.DataFrame:
    """
    根据efinance工具包获取股票数据
    :param stock_code:股票代码
    :param begin: 开始日期
    :param end: 结束日期
    :return:
    """
    # stock_code = '600519'  # 股票代码，茅台
    k_dataframe: pd.DataFrame = efinance.stock.get_quote_history(
        stock_code, beg=begin.strftime("%Y%m%d"), end=end.strftime("%Y%m%d")
    )
    k_dataframe = k_dataframe.iloc[:, :9]
    k_dataframe.columns = [
        "name",
        "code",
        "date",
        "open",
        "close",
        "high",
        "low",
        "volume",
        "turnover",
    ]
    k_dataframe.index = pd.to_datetime(k_dataframe.date)
    k_dataframe.drop(["name", "code", "date"], axis=1, inplace=True)
    return k_dataframe


class MyStrategy1(backtrader.Strategy):  # 策略
    def __init__(self):
        # 初始化交易指令、买卖价格和手续费
        self.close_price = self.datas[0].close  # 这里加一个数据引用，方便后续操作
        self.sma = backtrader.indicators.SimpleMovingAverage(
            self.datas[0], period=5
        )  # 借用这个策略，计算5日的均线
        self.order_list = []

    def notify_order(self, order):  # 固定写法，查看订单情况
        # 查看订单情况
        if order.status in [order.Submitted, order.Accepted]:  # 接受订单交易，正常情况
            return
        if order.status in [order.Completed]:
            if order.isbuy():
                logger.debug(
                    "{} 已买入, 购入金额 {} 手续费 {}",
                    order.ref,
                    order.executed.value,
                    order.executed.comm,
                )
            elif order.issell():
                logger.debug(
                    "{} 已卖出, 卖出金额 {} 手续费 {}",
                    order.ref,
                    order.executed.value,
                    order.executed.comm,
                )
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            logger.debug("{} 订单取消、保证金不足、金额不足拒绝交易", order.ref)
        elif order.status in [order.Expired]:
            logger.debug("订单 {} 超过时效，已取消", order.ref)

    def next(
        self,
    ):  # 固定的函数，框架执行过程中会不断循环next()，过一个K线，执行一次next()
        if self.close_price[0] > self.sma[0]:
            # 执行买入
            # 获得默认的保证金：self.broker.comminfo[None].p.commission
            order = self.buy(
                size=500,
                price=self.data.close[0],
                exectype=backtrader.Order.Limit,
                valid=timedelta(days=1),
            )
            logger.debug("订单：买 {} ", order.ref)
            self.order_list.append(order)
        # 执行卖出条件已有持仓，且收盘价格跌破5日均线
        if self.position:
            if self.close_price[0] < self.sma[0]:
                # 执行卖出
                order = self.sell(
                    size=500,
                    price=self.data.close[0],
                    exectype=backtrader.Order.Limit,
                    valid=timedelta(days=1),
                )
                logger.debug("订单：卖 {} ", order.ref)
                self.order_list.append(order)


if __name__ == "__main__":
    # 获取数据
    # start_time = datetime(2020, 1, 1)
    # end_time = datetime(2021, 1, 1)
    # dataframe = get_k_data('600519', begin=start_time, end=end_time)
    # # =============== 为系统注入数据 =================
    # # 加载数据
    # data = backtrader.feeds.PandasData(dataname=dataframe, fromdate=start_time, todate=end_time)
    store = QMTStore()
    code_list = ["600519.SH"]
    data = store.getdata(
        dataname="600519.SH",
        timeframe=bt.TimeFrame.Days,
        fromdate=datetime(2020, 1, 1),
        todate=datetime(2021, 1, 1),
        live=False,
    )
    # 初始化cerebro回测系统
    cerebral_system = (
        backtrader.Cerebro()
    )  # Cerebro引擎在后台创建了broker(经纪人)实例，系统默认每个broker的初始资金量为10000
    # 将数据传入回测系统
    cerebral_system.adddata(data)  # 导入数据，在策略中使用 self.datas 来获取数据源
    # 将交易策略加载到回测系统中
    cerebral_system.addstrategy(MyStrategy1)
    # =============== 系统设置 ==================
    # 设置启动资金为 100000
    start_cash = 1000000
    cerebral_system.broker.setcash(start_cash)
    # 设置手续费 万2.5
    cerebral_system.broker.setcommission(commission=0.00025)
    logger.debug(
        "初始资金: {} 回测期间：from {} to {}".format(start_cash, start_time, end_time)
    )
    # 运行回测系统
    cerebral_system.run()
    # 获取回测结束后的总资金
    portvalue = cerebral_system.broker.getvalue()
    pnl = portvalue - start_cash
    # 打印结果
    logger.debug("净收益: {}", pnl)
    logger.debug("总资金: {}", portvalue)

2025-04-29 12:01:45.660 | DEBUG    | __main__:<module>:89 - 初始资金: 1000000 回测期间：from 2020-01-01 00:00:00 to 2021-01-01 00:00:00
2025-04-29 12:01:45.750 | DEBUG    | __main__:next:53 - 订单：买 6848 
2025-04-29 12:01:45.751 | DEBUG    | __main__:notify_order:39 - 6848 已买入, 购入金额 551350.0 手续费 137.8375
2025-04-29 12:01:45.752 | DEBUG    | __main__:next:53 - 订单：买 6849 
2025-04-29 12:01:45.753 | DEBUG    | __main__:notify_order:43 - 6849 订单取消、保证金不足、金额不足拒绝交易
2025-04-29 12:01:45.753 | DEBUG    | __main__:next:53 - 订单：买 6850 
2025-04-29 12:01:45.754 | DEBUG    | __main__:notify_order:43 - 6850 订单取消、保证金不足、金额不足拒绝交易
2025-04-29 12:01:45.755 | DEBUG    | __main__:next:53 - 订单：买 6851 
2025-04-29 12:01:45.756 | DEBUG    | __main__:notify_order:43 - 6851 订单取消、保证金不足、金额不足拒绝交易
2025-04-29 12:01:45.756 | DEBUG    | __main__:next:53 - 订单：买 6852 
2025-04-29 12:01:45.757 | DEBUG    | __main__:notify_order:43 - 6852 订单取消、保证金不足、金额不足拒绝交易
2025-04-29 12:01:45.757 | DEBUG    | __main__:next:61 - 订单：卖 6853 
2025-04-29 12:

600519.SH历史数据装载成功！


2025-04-29 12:01:45.920 | DEBUG    | __main__:notify_order:43 - 6938 订单取消、保证金不足、金额不足拒绝交易
2025-04-29 12:01:45.921 | DEBUG    | __main__:next:53 - 订单：买 6939 
2025-04-29 12:01:45.922 | DEBUG    | __main__:notify_order:43 - 6939 订单取消、保证金不足、金额不足拒绝交易
2025-04-29 12:01:45.923 | DEBUG    | __main__:next:53 - 订单：买 6940 
2025-04-29 12:01:45.924 | DEBUG    | __main__:notify_order:43 - 6940 订单取消、保证金不足、金额不足拒绝交易
2025-04-29 12:01:45.925 | DEBUG    | __main__:next:53 - 订单：买 6941 
2025-04-29 12:01:45.927 | DEBUG    | __main__:notify_order:43 - 6941 订单取消、保证金不足、金额不足拒绝交易
2025-04-29 12:01:45.928 | DEBUG    | __main__:next:53 - 订单：买 6942 
2025-04-29 12:01:45.929 | DEBUG    | __main__:notify_order:43 - 6942 订单取消、保证金不足、金额不足拒绝交易
2025-04-29 12:01:45.930 | DEBUG    | __main__:next:53 - 订单：买 6943 
2025-04-29 12:01:45.931 | DEBUG    | __main__:notify_order:43 - 6943 订单取消、保证金不足、金额不足拒绝交易
2025-04-29 12:01:45.932 | DEBUG    | __main__:next:53 - 订单：买 6944 
2025-04-29 12:01:45.933 | DEBUG    | __main__:notify_order:43 - 

### Train和Test流程demo

In [17]:
from datetime import datetime
import backtrader as bt
import pandas as pd
import efinance
import optuna


def get_k_data(stock_code, begin: datetime, end: datetime) -> pd.DataFrame:
    """根据efinance工具包获取股票数据
    :param stock_code:股票代码
    :param begin: 开始日期
    :param end: 结束日期
    """
    # stock_code = '600519'  # 股票代码，茅台
    k_dataframe: pd.DataFrame = efinance.stock.get_quote_history(
        stock_code, beg=begin.strftime("%Y%m%d"), end=end.strftime("%Y%m%d")
    )
    k_dataframe = k_dataframe.iloc[:, :9]
    k_dataframe.columns = [
        "name",
        "code",
        "date",
        "open",
        "close",
        "high",
        "low",
        "volume",
        "turnover",
    ]
    k_dataframe.index = pd.to_datetime(k_dataframe.date)
    k_dataframe.drop(["name", "code", "date"], axis=1, inplace=True)
    return k_dataframe


class SmaCross(bt.SignalStrategy):
    params = (
        ("period1", 10),
        ("period2", 30),
    )

    def __init__(self):
        sma1, sma2 = bt.ind.SMA(period=self.p.period1), bt.ind.SMA(
            period=self.p.period2
        )
        crossover = bt.ind.CrossOver(sma1, sma2)
        self.signal_add(bt.SIGNAL_LONG, crossover)


class backtest:
    def __init__(self, data_list, strategy):
        self.data_list = data_list
        self.strategy = strategy
        self.cerebro = bt.Cerebro()
        if isinstance(data_list, list):
            for data in data_list:
                self.cerebro.adddata(data)
        else:
            self.cerebro.adddata(data_list)
        self.start_cash = 1000000
        self.cerebro.broker.setcash(self.start_cash)
        self.cerebro.broker.setcommission(commission=0.00025)

    def __call__(self, trial):
        p1 = trial.suggest_int("period1", 1, 50)
        p2 = trial.suggest_int("period2", 1, 50)
        self.cerebro.strats = []
        self.cerebro.addstrategy(self.strategy, period1=int(p1), period2=int(p2))
        self.cerebro.run()
        return self.start_cash - self.cerebro.broker.getvalue()


def train(data_list, strategy):
    study = optuna.create_study(direction="maximize")
    study.optimize(
        backtest(data_list=data_list, strategy=strategy), n_trials=10, n_jobs=1
    )
    trial = study.best_trial

    return study.best_trial.value, study.best_trial.params


def test(data_list, strategy, *args, **kwargs):
    data_list = data_list
    strategy = strategy
    cerebro = bt.Cerebro()
    if isinstance(data_list, list):
        for data in data_list:
            cerebro.adddata(data)
    else:
        cerebro.adddata(data_list)
    start_cash = 1000000
    cerebro.broker.setcash(start_cash)
    cerebro.broker.setcommission(commission=0.00025)

    cerebro.addstrategy(strategy, *args, **kwargs)
    cerebro.run()
    return start_cash - cerebro.broker.getvalue()


#### Train
# start_time = datetime(2020, 1, 1)
# end_time = datetime(2021, 1, 1)
# dataframe = get_k_data('600519', begin=start_time, end=end_time)
# data = bt.feeds.PandasData(dataname=dataframe, fromdate=start_time, todate=end_time)
store = QMTStore()
code_list = ["600519.SH"]
data = store.getdata(
    dataname="600519.SH",
    timeframe=bt.TimeFrame.Days,
    fromdate=datetime(2020, 1, 1),
    todate=datetime(2021, 1, 1),
    live=False,
)
value, params = train(data, SmaCross)
print("train value: ", value)


#### Test
# start_time = datetime(2021, 1, 1)
# end_time = datetime(2022, 1, 1)
# dataframe = get_k_data('600519', begin=start_time, end=end_time)
# data = bt.feeds.PandasData(dataname=dataframe, fromdate=start_time, todate=end_time)
store = QMTStore()
code_list = ["600519.SH"]
data = store.getdata(
    dataname="600519.SH",
    timeframe=bt.TimeFrame.Days,
    fromdate=datetime(2020, 1, 1),
    todate=datetime(2021, 1, 1),
    live=False,
)
print("test value: ", test(data, SmaCross, **params))

[I 2025-04-29 12:05:16,157] A new study created in memory with name: no-name-65cab7b5-4423-4b28-9e09-33fe4e2bc392
[I 2025-04-29 12:05:16,281] Trial 0 finished with value: -774.4969474999234 and parameters: {'period1': 14, 'period2': 47}. Best is trial 0 with value: -774.4969474999234.
[I 2025-04-29 12:05:16,378] Trial 1 finished with value: -743.6604049999733 and parameters: {'period1': 4, 'period2': 41}. Best is trial 1 with value: -743.6604049999733.


600519.SH历史数据装载成功！
600519.SH历史数据装载成功！
600519.SH历史数据装载成功！


[I 2025-04-29 12:05:16,476] Trial 2 finished with value: -223.87076250032987 and parameters: {'period1': 28, 'period2': 25}. Best is trial 2 with value: -223.87076250032987.
[I 2025-04-29 12:05:16,595] Trial 3 finished with value: 91.35541499999817 and parameters: {'period1': 42, 'period2': 36}. Best is trial 3 with value: 91.35541499999817.
[I 2025-04-29 12:05:16,702] Trial 4 finished with value: -631.1280349999433 and parameters: {'period1': 18, 'period2': 28}. Best is trial 3 with value: 91.35541499999817.


600519.SH历史数据装载成功！
600519.SH历史数据装载成功！


[I 2025-04-29 12:05:16,812] Trial 5 finished with value: -543.9177599998657 and parameters: {'period1': 1, 'period2': 35}. Best is trial 3 with value: 91.35541499999817.
[I 2025-04-29 12:05:16,919] Trial 6 finished with value: -234.8400849996833 and parameters: {'period1': 29, 'period2': 28}. Best is trial 3 with value: 91.35541499999817.


600519.SH历史数据装载成功！
600519.SH历史数据装载成功！


[I 2025-04-29 12:05:17,022] Trial 7 finished with value: -660.1155199999921 and parameters: {'period1': 5, 'period2': 50}. Best is trial 3 with value: 91.35541499999817.
[I 2025-04-29 12:05:17,127] Trial 8 finished with value: -655.0870325000724 and parameters: {'period1': 18, 'period2': 27}. Best is trial 3 with value: 91.35541499999817.


600519.SH历史数据装载成功！
600519.SH历史数据装载成功！


[I 2025-04-29 12:05:17,230] Trial 9 finished with value: -446.77689749968704 and parameters: {'period1': 6, 'period2': 16}. Best is trial 3 with value: 91.35541499999817.


600519.SH历史数据装载成功！
train value:  91.35541499999817
600519.SH历史数据装载成功！
test value:  91.35541499999817
